# Ranking das Bibs dos Repositórios do Dataset

Com os dados separados obter e identificar quais dependências aparecem com mais frequência.

## Importação de Bibliotecas

- **pandas (`pd`)**: Para manipulação de dados em formato tabular.
- **json**: Trabalhar com dados no formato JSON
- **collections**: Contar ocorrências e a biblioteca

## Desenvolvimento 

Para fazer essa contagem foi feito códigos para cada tipo de arquivo de configuração, em particular para os `package.json`, `composer.json`, `angular.json`, `tsconfig.json`

# Para o `Package.json`

1. **Carregamento de Dados**: O script carrega um arquivo CSV que contém o conteúdo de arquivos `package.json`.

2. **Contagem de Dependências**: Utiliza um contador para registrar a frequência de cada dependência nas seções `dependencies` e `devDependencies` dos arquivos JSON.

3. **Processamento de Dados**: Para cada linha do DataFrame:
   - Tenta carregar o conteúdo da coluna `content` como um objeto JSON.
   - Extrai as dependências e dev dependências, atualizando o contador com os nomes dos plugins.

4. **Geração de Relatório**: Converte o contador de dependências em um DataFrame e salva os resultados em um novo arquivo CSV, que lista todos os plugins e suas respectivas frequências.


In [ ]:
import pandas as pd
from collections import Counter
import json

# Carregar o arquivo package.json.csv
# Substitua pelo caminho correto
file_path = '../Database/Questao1/separados/package.json.csv'
data = pd.read_csv(file_path)

# Inicializar um contador para os plugins
plugin_counts = Counter()

# Processar cada linha da coluna 'content' como JSON
for index, row in data.iterrows():
    try:
        # Tentar carregar cada linha como JSON
        package_data = json.loads(row['content'])

        # Extrair plugins das seções 'dependencies' e 'devDependencies'
        dependencies = package_data.get('dependencies', {})
        dev_dependencies = package_data.get('devDependencies', {})

        # Adicionar plugins ao contador
        plugin_counts.update(dependencies.keys())
        plugin_counts.update(dev_dependencies.keys())
    except json.JSONDecodeError as e:
        print(f"Erro ao carregar JSON na linha {index}: {e}")

# Converter para DataFrame para visualizar todos os plugins e suas frequências
plugin_list = pd.DataFrame(plugin_counts.items(), columns=[
                           'Plugin', 'Frequency'])

plugin_list.to_csv('../Database/Questao1/Ranking/package.csv', index=False)
# Exibir todos os plugins com a quantidade de vezes que aparecem
print(plugin_list)

                               Plugin  Frequency
0      @babel/plugin-external-helpers          4
1     @babel/plugin-transform-runtime         43
2              @storybook/preset-scss          2
3                           cross-env         97
4                            gh-pages         29
...                               ...        ...
6155                         vue-tour          1
6156                vue2-touch-events          1
6157       vuetify-draggable-treeview          1
6158                      webm-to-mp4          1
6159                           xstate          1

[6160 rows x 2 columns]


# Para o `Angular.json`

1. **Carregamento de Dados**: O script carrega um arquivo CSV que contém o conteúdo de arquivos `angular.json`.

2. **Extração de Informações dos Projetos**: Para cada linha do DataFrame:
   - Tenta carregar o conteúdo da coluna `content` como um objeto JSON.
   - Verifica se a chave `projects` existe e, se sim, itera sobre cada projeto, extraindo informações relevantes como:
     - Nome do projeto
     - Tipo do projeto
     - Raiz do projeto
     - Raiz da fonte

3. **Criação de DataFrame**: As informações extraídas são armazenadas em uma lista, que é convertida em um DataFrame.

4. **Geração de Relatórios**:
   - Salva as informações dos projetos em um arquivo CSV.
   - Conta a frequência dos nomes dos projetos e salva esses dados em outro arquivo CSV.


In [3]:

# Carregar o arquivo angular.json.csv
file_path = '../Database/Questao1/separados/angular.json.csv'
data = pd.read_csv(file_path)

# Inicializar uma lista para armazenar as informações dos projetos
projects_info = []

# Processar cada linha da coluna 'content' como JSON
for index, row in data.iterrows():
    try:
        angular_data = json.loads(row['content'])

        # Verificar se a chave 'projects' existe
        if 'projects' in angular_data:
            projects = angular_data['projects']

            # Iterar sobre cada projeto
            for project_name, project_data in projects.items():
                # Extrair informações relevantes de cada projeto
                project_info = {
                    'project_name': project_name,
                    'project_type': project_data.get('projectType', 'N/A'),
                    'root': project_data.get('root', 'N/A'),
                    'sourceRoot': project_data.get('sourceRoot', 'N/A'),
                    # Adicione outras informações relevantes aqui
                }
                projects_info.append(project_info)
    except json.JSONDecodeError as e:
        print(f"Erro ao carregar JSON na linha {index}: {e}")

# Criar um DataFrame com as informações dos projetos
projects_df = pd.DataFrame(projects_info)

# Salvar as informações dos projetos em um arquivo CSV
projects_df.to_csv('../Database/Questao1/Ranking/angular_projects.csv', index=False)

# Exibir as informações dos projetos
print(projects_df)

# Contar a frequência dos nomes de projetos
project_name_counts = Counter(project['project_name'] for project in projects_info)

# Criar um DataFrame com as informações de frequência
project_name_freq = pd.DataFrame.from_dict(project_name_counts, orient='index', columns=['Frequência'])

# Resetar o índice para tornar a coluna 'project_name' uma coluna normal
project_name_freq = project_name_freq.reset_index().rename(columns={'index': 'project_name'})

# Salvar o DataFrame em um arquivo CSV
project_name_freq.to_csv('../Database/Questao1/Ranking/angular_analysis.csv', index=False)

# Exibir os resultados
print(project_name_freq)

                                      project_name project_type  \
0                                ng-zorro-antd-doc  application   
1                             ng-zorro-antd-iframe  application   
2                                ng-zorro-antd-lib      library   
3                                       playground  application   
4                                   playground-e2e  application   
5                                             docs  application   
6                                            theme      library   
7                                             auth      library   
8                                         date-fns      library   
9                                        eva-icons      library   
10                                   firebase-auth      library   
11                                          moment      library   
12                                        security      library   
13                                           devui  applicatio

# Para o `tsconfig.json`

1. **Carregamento de Dados**: O script carrega um arquivo CSV que contém o conteúdo de arquivos `tsconfig.json`.

2. **Contagem de Configurações**: Inicializa um contador para armazenar as frequências de cada configuração em `compilerOptions`. Para cada linha do DataFrame:
   - Verifica se a coluna `content` existe e se é uma string JSON válida.
   - Tenta carregar o conteúdo da coluna `content` como um objeto JSON.
   - Extrai o objeto `compilerOptions` e conta as configurações, ignorando a chave `paths`.

3. **Tratamento de Tipos**: O script trata diferentes tipos de valores nas configurações:
   - Para valores do tipo string ou booleano, conta diretamente.
   - Para valores do tipo lista, conta cada item individualmente.

4. **Geração de Relatório**: Converte o contador de configurações em um DataFrame e salva os resultados em um arquivo CSV.


In [4]:

# Carregar o arquivo tsconfig.json.csv
file_path = '../Database/Questao1/separados/tsconfig.json.csv'
data = pd.read_csv(file_path)

# Inicializar um Counter para cada configuração em compilerOptions
compiler_options_counts = Counter()

# Processar cada linha da coluna 'content' como JSON
for index, row in data.iterrows():
    try:
        # Verificar se a coluna 'content' existe e é uma string JSON válida
        if 'content' in row and isinstance(row['content'], str) and row['content'].strip():
            tsconfig_data = json.loads(row['content'])

            # Obter o objeto compilerOptions
            compiler_options = tsconfig_data.get('compilerOptions', {})

            # Contar configurações que são valores únicos, ignorando 'paths'
            for key, value in compiler_options.items():
                if key == 'paths':  # Ignorar a chave 'paths'
                    continue

                if isinstance(value, (str, bool)):  # Se o valor for uma string ou booleano
                    compiler_options_counts[(key, value)] += 1
                elif isinstance(value, list):  # Se o valor for uma lista (como 'lib')
                    for item in value:
                        compiler_options_counts[(key, item)] += 1
                # Ignorar dicionários como 'paths'

    except json.JSONDecodeError as e:
        print(f"Erro ao carregar JSON na linha {index}: {e}")
    except TypeError as e:
        print(f"Erro ao processar dados na linha {index}: {e}")

# Criar um DataFrame com as contagens de configurações do compilerOptions
compiler_options_freq = pd.DataFrame(
    compiler_options_counts.items(), 
    columns=['Configuração', 'Frequência']
)

# Salvar o DataFrame em um arquivo CSV
compiler_options_freq.to_csv('../Database/Questao1/Ranking/tsconfig.csv', index=False)

print("Contagem de configurações concluída e salva em 'tsconfig.csv'")

Erro ao carregar JSON na linha 5: Expecting value: line 1 column 1 (char 0)
Erro ao carregar JSON na linha 8: Expecting value: line 46 column 3 (char 955)
Erro ao carregar JSON na linha 9: Expecting value: line 1 column 1 (char 0)
Erro ao carregar JSON na linha 10: Expecting property name enclosed in double quotes: line 19 column 9 (char 715)
Erro ao carregar JSON na linha 11: Expecting value: line 1 column 1 (char 0)
Erro ao carregar JSON na linha 13: Expecting ',' delimiter: line 35 column 17 (char 824)
Erro ao carregar JSON na linha 16: Expecting property name enclosed in double quotes: line 3 column 5 (char 29)
Erro ao carregar JSON na linha 19: Expecting value: line 1 column 1 (char 0)
Erro ao processar dados na linha 22: unhashable type: 'dict'
Erro ao processar dados na linha 23: unhashable type: 'dict'
Erro ao processar dados na linha 30: unhashable type: 'dict'
Erro ao carregar JSON na linha 31: Expecting property name enclosed in double quotes: line 4 column 5 (char 92)
Erro 

# Para o `Composer.json`

1. **Carregamento de Dados**: O script carrega um arquivo CSV que contém o conteúdo de arquivos `composer.json`.

2. **Contagem de Dependências**: Inicializa contadores para armazenar as frequências de pacotes nas seções `require` e `require-dev`. Para cada linha do DataFrame:
   - Verifica se a coluna `content` existe e se é uma string JSON válida.
   - Tenta carregar o conteúdo da coluna `content` como um objeto JSON.
   - Conta os pacotes na seção `require` e na seção `require-dev`.

3. **Tratamento de Erros**: O script inclui tratamento de erros para lidar com falhas na leitura do JSON e problemas de tipo.

4. **Geração de Relatórios**: Cria DataFrames para armazenar as frequências das dependências e salva os resultados em arquivos CSV separados.

In [6]:

# Carregar o arquivo composer.json.csv
file_path = '../Database/Questao1/separados/composer.json.csv'
data = pd.read_csv(file_path)

# Inicializar contadores para dependências e outras informações relevantes
require_counts = Counter()
require_dev_counts = Counter()
autoload_counts = Counter()

# Processar cada linha da coluna 'content' como JSON
for index, row in data.iterrows():
    try:
        # Verificar se a coluna 'content' existe e é uma string JSON válida
        if 'content' in row and isinstance(row['content'], str) and row['content'].strip():
            composer_data = json.loads(row['content'])

            # Contar pacotes na seção 'require'
            if 'require' in composer_data:
                for package, version in composer_data['require'].items():
                    require_counts[package] += 1

            # Contar pacotes na seção 'require-dev'
            if 'require-dev' in composer_data:
                for package, version in composer_data['require-dev'].items():
                    require_dev_counts[package] += 1


    except json.JSONDecodeError as e:
        print(f"Erro ao carregar JSON na linha {index}: {e}")
    except TypeError as e:
        print(f"Erro ao processar dados na linha {index}: {e}")

# Criar DataFrames para salvar as frequências das dependências e autoload
require_df = pd.DataFrame(require_counts.items(), columns=['Pacote', 'Frequência'])
require_dev_df = pd.DataFrame(require_dev_counts.items(), columns=['Pacote Dev', 'Frequência'])

# Salvar cada DataFrame em arquivos CSV separados
require_df.to_csv('../Database/Questao1/Ranking/composer_require_freq.csv', index=False)
require_dev_df.to_csv('../Database/Questao1/Ranking/composer_require_dev_freq.csv', index=False)

print("Contagem de dependências e configurações autoload concluída e salva nos arquivos CSV.")

Contagem de dependências e configurações autoload concluída e salva nos arquivos CSV.
